In [ ]:
# Import the necessary libraries
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Create a function to build an autoencoder
def build_autoencoder(input_dim, hidden_dim):
    # Input layer
    input_layer = layers.Input(shape=(input_dim,))
    # Encoding layer
    encoded = layers.Dense(hidden_dim, activation='relu', name='encoder')(input_layer)
    # Decoding layer
    decoded = layers.Dense(input_dim, activation='sigmoid')(encoded)
    # Creating autoencoder model
    autoencoder = keras.Model(input_layer, decoded)
    # Compiling the model
    autoencoder.compile(optimizer='adam', loss='mse')
    # Displaying the model summary
    autoencoder.summary()
    return autoencoder

In [ ]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading the imaging dataset
data = np.load('/content/drive/MyDrive/Emotion Detection/X_test_Happy.npy')
print(data.shape)
# Flattening the images
data = data.reshape((len(data), np.prod(data.shape[1:])))
print(data.shape)

In [ ]:
# Choose the size of hidden dimensions and train the model
hidden_dims = [1024, 512]
encoder_model = None
for i, hidden_dim in enumerate(hidden_dims):
    if i == 0:
        # Building the initial autoencoder model
        autoencoder = build_autoencoder(data.shape[1], hidden_dim)
    else:
        # Building subsequent autoencoder models with previous encoder output as input
        autoencoder = build_autoencoder(encoder_model.output_shape[1], hidden_dim)
    # Training the autoencoder model
    autoencoder.fit(data, data, epochs=10, batch_size=32)
    # Creating encoder model to get the encoded output
    encoder_model = keras.Model(autoencoder.input, autoencoder.get_layer('encoder').output)
    # Encoding the data
    data = encoder_model.predict(data)
    final_encoder = encoder_model
print(data.shape)